<a href="https://colab.research.google.com/github/davidzimniok/MPA-MLF/blob/main/projekt02/PROJEKT02_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from PIL import Image #library for loading png files
import glob           #library for multiple loading
import csv            #library for loading csv file
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping
from keras.optimizers import Adamax
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import re
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split

In [ ]:
#mount Gdrive and load train, test dataset and labels
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# UNZIP files on gdrive  !RUN only once!


In [ ]:
# Path to the ZIP file on Google Drive
zip_file_path = '/content/drive/MyDrive/Colab Notebooks/projekt02/train_data_unlabeled.zip'

# Path to the folder where you want to extract the files
extracted_folder_path = '/content/drive/MyDrive/Colab Notebooks/projekt02'

# Unzip the file
!unzip "$zip_file_path" -d "$extracted_folder_path"

print("Extraction completed successfully.")

In [ ]:
# Path to the ZIP file on Google Drive
zip_file_path = '/content/drive/MyDrive/Colab Notebooks/projekt02/test_data_unlabeled.zip'

# Path to the folder where you want to extract the files
extracted_folder_path = '/content/drive/MyDrive/Colab Notebooks/projekt02'

# Unzip the file
!unzip "$zip_file_path" -d "$extracted_folder_path"

print("Extraction completed successfully.")

# Load test and train datasets

In [ ]:
#function to get number of measure from the path
def extract_number(filename):
    filename = filename[48:]
    return int(re.search(r'\d+', filename).group())

#function for loading dataset from Gdrive
def load_dataset(path):
  data=[]

  file_paths=glob.glob(path + "/*.png")
  file_paths=sorted(file_paths, key=extract_number)

  for im_path in file_paths:
      im = Image.open(im_path).convert('L')
      data.append(im)

  image_arr=[np.array(img) for img in data]
  image_arr=np.array(image_arr)/255.0
  return image_arr

train_data=load_dataset("/content/drive/MyDrive/Colab Notebooks/projekt02/train_data_unlabeled")
test_data=load_dataset("/content/drive/MyDrive/Colab Notebooks/projekt02/test_data_unlabeled")

labels=[]
with open("/content/drive/MyDrive/Colab Notebooks/projekt02/y_train.csv", 'r', newline='', encoding='utf-8') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',')
    next(csvreader) #skip header of column
    #save data to the list
    for row in csvreader:
        labels.append(row[1])

y_labels=np.array(labels)
y_labels = to_categorical(y_labels, num_classes=4)

In [ ]:
#data augmentation - not used
X_train, X_test, y_train, y_test = train_test_split(train_data, y_labels, test_size=0.2)

X_train = X_train.reshape(-1, 45, 51, 1)
X_test = X_test.reshape(-1, 45, 51, 1)

datagen = ImageDataGenerator(featurewise_center=False,
                            samplewise_center=False,
                            featurewise_std_normalization=False,
                            samplewise_std_normalization=False,
                            zca_whitening=False,
                            rotation_range=1,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            horizontal_flip=False,
                            vertical_flip=False)

datagen.fit(X_train)

#logaritmic scaling
train_data = np.log(train_data + 1)
train_data = train_data / np.max(train_data)

# Create NN model and train the network

In [3]:
#create NN model

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(45, 51, 1)))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='sigmoid', kernel_initializer='he_uniform'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='sigmoid', kernel_initializer='he_uniform'))
model.add(Dropout(0.3))
model.add(Dense(128, activation='sigmoid', kernel_initializer='he_uniform'))
model.add(Dropout(0.4))
model.add(Dense(64, activation='sigmoid', kernel_initializer='he_uniform'))

model.add(Dense(4, activation='softmax'))

In [4]:
optimizer=Adamax(learning_rate=0.001)
#model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])
early_stopping=EarlyStopping(monitor='val_accuracy',patience=5,restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

NameError: name 'ReduceLROnPlateau' is not defined

In [ ]:
history=model.fit(train_data,y_labels,validation_split=0.2,epochs=100,callbacks=[early_stopping,reduce_lr])

In [ ]:
#fitting with augmented data - not used
history = model.fit(datagen.flow(X_train, y_train),  # Training data with data augmentation
                    #validation_data=datagen.flow(X_test.astype('float32') / 255.0, to_categorical(y_test), batch_size=512),
                    validation_data=datagen.flow(X_test, y_test),
                    epochs=30)

In [ ]:
#evaluate the model accurancy
score=model.evaluate(train_data,y_labels,verbose=0)
print('Test loss:',score[0])
print(f'Test accuracy: {score[1]*100} %')

In [ ]:
#plot loss and accurancy
plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.figure()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

# Save trained network and predict data

In [ ]:
path_model="model_09042024.h5"

#save trained model to the Gdrive
model.save('/content/drive/MyDrive/Colab Notebooks/projekt02/models/'+path_model)
print("Model saved to Google Drive.")

#load model from Gdrive
#model=load_model(model_path)

Model saved to Google Drive.


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
#apply model to the provided dataset
predictions=model.predict(test_data)

150/150 [==============================] - 13s 87ms/step


In [ ]:
#convert predictions (probability of number of persons in room) to the index = number of persons
predicted=np.argmax(predictions,axis=1)

prediction_file="model_09042024_1.csv"

csv_file_path='/content/drive/MyDrive/Colab Notebooks/projekt02/outputs/'+prediction_file
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['id', 'target'])
    for idx, prediction in enumerate(predicted):
        writer.writerow([idx, prediction])

print(f"Predictions saved to CSV file: {csv_file_path}")

Predictions saved to CSV file: /content/drive/MyDrive/Colab Notebooks/projekt02/model_09042024_1.csv
